In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.path as mplPath
#from shapely.geometry import Point
# shapely.geometry.polygon import Polygon

class SailingEnv(gym.Env):
    def __init__(self, max_steps=20_000):
        self.action_space = gym.spaces.Discrete(4) #0=west, 1=north or 2=east, 3=south
        self.minX=self.minY=0#.0
        self.maxX=self.maxY=19#.0
        #self.observation_space = gym.spaces.Box(low=np.array([minX, minY]), high=np.array([maxX, maxY]), dtype=np.float32)
        #self.wind_direction = np.array([1.0, 0.0])
        #self.rudder_angle = 0
        self.x = 0#.0
        self.y = 0#.0
        self.position = np.array([self.x, self.y])
        self.max_steps = max_steps
        self.current_step = 0
        self.target = np.array([8, 9])

    def reset(self):
        #self.wind_direction = np.array([1.0, 0.0])
        #self.rudder_angle = 0
        #self.position = np.array([self.x, self.y])
        self.position =  np.random.randint(self.minX//2, self.maxX//2, size=(1, 2))[0]
        self.current_step = 0
        #return self.position
        return tuple(self.position.astype(np.int64))
    
    def move():
        pass

    def step(self, action):
        #meta to step prepei na epistrepsei:
        #new_state (edw x,y), reward, done , {} 
        assert self.action_space.contains(action)
        self.current_step += 1
        if action==0:
            self.position[0]-=1
        elif action==1:
            self.position[1]+=1
        elif action==2:
            self.position[0]+=1
        else:
            self.position[1]-=1    
            
        #self.position=np.array([1,0])    
        #checks if position after action is contained in our grid
#         self.boundaries = [[self.minX-1,self.minY-1], 
#                            [self.maxX+1,self.minY-1], 
#                            [ self.maxX+1, self.maxY+1], 
#                            [self.minX-1,self.maxY+1]]
#         self.polygon=mplPath.Path(np.array(self.boundaries))
#         self.in_grid_condition = self.polygon.contains_point(self.position)
        #return self.in_grid_condition, self.position, self.boundaries
        
#         if self.position[0] < 0  or self.position
        #if not self.in_grid_condition:
            #reward=-200
        if (self.position==self.target).all():
            reward=1
        else:
            reward=-0.5
            
        #if self.position[0] >= maxX or self.position[1] >= maxY or self.position[0] < minX or self.position[1] < minY
        

#         reward = 1 if (self.position==self.target).all() else -0.8
#         #reward = np.cos(boat_direction - np.arctan2(self.target[1] - self.position[1], self.target[0] - self.position[0]))
        
        truncated = self.position[0] > self.maxX or self.position[1] > self.maxY or self.position[0] < self.minX or self.position[1] < self.minY  or self.current_step >= self.max_steps
        #or (self.position==self.target).all()
        done = (self.position==self.target).all() 
        
        return tuple(self.position.astype(np.int64)), reward, done, truncated

env = SailingEnv()



In [2]:
#q_table will have 10X10X4 cells
q_table = np.random.uniform(low=-2, high=0, size=([20,20] + [env.action_space.n]))
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPSILON=0.9

In [3]:
# Exploration settings
EPISODES=100_000
EPSILON = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//4
epsilon_decay_value = EPSILON/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

In [4]:
q_table = np.random.uniform(low=-2, high=0, size=([20,20] + [env.action_space.n]))
#all_actions=[]
success=-1
for i in range(EPISODES):    
    done=False
    current_state=env.reset()
    ep_reward=0    
    while not done:
        if np.random.random() < EPSILON:
            action=env.action_space.sample()
        else:
            action=np.argmax(q_table[current_state]) #best action based on q-values
        #print(f"action={action}")
        #all_actions.append(action)
        new_state, reward, done, truncated = env.step(action)
        if truncated:
            #poly mikro Q value otan o agent odhgeitai eksw apo to grid
            #gia na mhn ksanaepileksei tetoia action otan vrethei sthn idia thesh
            #q_table[current_state+(action,)]=-20
            break
        else:
            #Q_max
            Q_max = np.max(q_table[new_state])
            #Q_current
            Q_current = q_table[current_state + (action,)]
            #new_q
            new_q = (1-LEARNING_RATE)*Q_current + LEARNING_RATE*(reward+DISCOUNT*Q_max)
            #update q_table
            q_table[current_state+(action,)]=new_q
            #replace new_state as current_state
            ep_reward+=reward
        #print(done)
        if done:
            success+=1
            #print("eftasa")
            
#         print(f"{env.current_step} step of {i} ep.,CurState={current_state},New Pos={new_state},RWD={reward},EP_RWD={ep_reward}, D={done}, T={truncated}, New Q-value={new_q} ")
        current_state=new_state
    #print("-====================================================================-\ ")
          #\n-====================================================================-")
        if END_EPSILON_DECAYING >= i >= START_EPSILON_DECAYING:
            EPSILON -= epsilon_decay_value
if success<0:
    print("Den oloklhrwsa kanena epeisodio.")
else:
    print(f"eftasa to {100*success/i}% twn epeisodiwn")


eftasa to 25.883258832588325% twn epeisodiwn


In [5]:
success/i

0.25883258832588324

## Elegxos tou q_table (oxi pia training)

In [6]:
success=-1
for i in range(1000):    
    done=False
    current_state=env.reset()       
    while not done:
        action=np.argmax(q_table[current_state]) #best action based on q-values       
        new_state, reward, done, truncated = env.step(action)
        #print(new_state)
        #print(env.current_step)
        if done:
            success+=1
            #print("eftasa")
        if truncated:
            break
        current_state=new_state
    #print("==")
if success<0:
    print("Den oloklhrwsa kanena epeisodio.")
else:
    print(f"eftasa to {100*success/i}% twn epeisodiwn")

eftasa to 22.52252252252252% twn epeisodiwn
